In [30]:
from langchain_groq import ChatGroq

In [31]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [32]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-38844?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Software Engineer I

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer IBeaverton, OregonBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does more than

In [33]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [34]:
print(res.content)

```json
{
  "role": "Software Engineer I",
  "experience": "2+ years of professional experience in application engineering",
  "skills": [
    "JavaScript (Node.js, React.js, Vue.js, etc.)",
    "Agile processes",
    "Continuous integration",
    "Test automation",
    "AWS CloudFront",
    "AWS S3",
    "AWS Route 53",
    "AWS Certificate Manager",
    "AWS Lambda",
    "GitHub Co-pilot"
  ],
  "description": "The Digital Employee Experience Software Engineer I will be responsible for design, support, development, and testing of internal employee-facing websites and web applications."
}
```


In [11]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer I',
 'experience': '2+ years of professional experience in application engineering',
 'skills': ['JavaScript (Node.js, React.js, Vue.js, etc.)',
  'Agile processes',
  'Continuous integration',
  'Test automation',
  'AWS CloudFront',
  'AWS S3',
  'AWS Route 53',
  'AWS Certificate Manager',
  'AWS Lambda',
  'GitHub Co-pilot'],
 'description': 'The Digital Employee Experience Software Engineer I will be responsible for design, support, development, and testing of internal employee-facing websites and web applications.'}

In [12]:
type(json_res)

dict

In [16]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,AI and ML,https://www.virtusa.com/services/engineer-auto...
1,Application Development,https://www.virtusa.com/services/engineer-auto...
2,API and Micro-services,https://www.virtusa.com/services/engineer-auto...
3,Frontend Engineering,https://www.virtusa.com/services/engineer-auto...
4,cloud,https://www.virtusa.com/services/engineer-auto...
5,Data Engineering,https://www.virtusa.com/services/engineer-auto...
6,DEVOPS AND SECOPS,https://virtusa.com/services/engineer-automate...
7,Analytics,https://www.virtusa.com/services/analyze-and-o...
8,BlockChain,https://www.virtusa.com/services/engineer-auto...
9,Generative AI,https://www.virtusa.com/services/engineer-auto...


In [22]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [38]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://www.virtusa.com/services/engineer-automate/application-engineering/frontend-engineering'},
  {'links': 'https://www.virtusa.com/services/engineer-automate/application-engineering/application-development'}],
 [{'links': 'https://www.virtusa.com/services/engineer-automate/application-engineering/application-development'},
  {'links': 'https://www.virtusa.com/services/consult-and-design/experience-design/design-ops'}],
 [{'links': 'https://www.virtusa.com/services/analyze-and-optimize/analytics/mlops'},
  {'links': 'https://virtusa.com/services/engineer-automate/delivery-transformation/enterprise-devsecops'}],
 [{'links': 'https://www.virtusa.com/services/engineer-automate/data-engineering/data-modeling'},
  {'links': 'https://www.virtusa.com/services/engineer-automate/application-engineering/frontend-engineering'}],
 [{'links': 'https://www.virtusa.com/services/engineer-automate/cloud/cloud-business-transformation'},
  {'links': 'https://www.virtusa.com/services/engi

In [37]:
job = json_res
job['skills']

['JavaScript (Node.js, React.js, Vue.js, etc.)',
 'Agile processes',
 'Continuous integration',
 'Test automation',
 'AWS CloudFront',
 'AWS S3',
 'AWS Route 53',
 'AWS Certificate Manager',
 'AWS Lambda',
 'GitHub Co-pilot']

In [39]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        Generating an cold email where we can find projects for virtusa by passing job description and links to virtusa services.
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Collaboration Opportunity for Custom Software Development with Virtusa

Dear [Hiring Manager's Name],

I came across the job description for a Software Engineer I at [Company Name] and was impressed by the exciting projects your team is working on. As a representative of Virtusa, a leading global technology consulting and services company, I'd like to explore potential collaboration opportunities that can help you achieve your goals.

Our team at Virtusa specializes in designing, developing, and delivering innovative software solutions that meet the evolving needs of businesses like yours. We noticed that the job description mentions the use of technologies such as Node.js, React.js, Vue.js, AWS CloudFront, AWS S3, and AWS Lambda, which align with our expertise.

We'd like to introduce you to our custom software development services, which can help you accelerate your project timelines and reduce costs. Our team can assist with:

* Designing and developing internal employee-fa